# Build Topic Model
The present notebook will use the data collected in section 5.1 to compute and explore a topic model. In essence, a topic model consists of two sets of probability distributions. First, the probability that a *word* belongs to a certain topic (Topic/Term probability). Second, the probability of a *topic* to appear in a document (Document/Topic probability). These probabilities are never 0, that is, each word has some probability (even if very small) to be part of each topic and each topic has some probability (even if very small) to appear in a document.

A standard way to inspect a topic model is to look at the top-ten words of a topic (the ten words with the highest probability in each particular topic). Similarly, we can pull the ten most important documents for each topic (the documents in which this topic has the highest probability).

For a more thorough analysis we may create full probability tables: a topic/term probability table and a document/topic probability table. These tables give a fuller account of the model and will be used for the visualizations (5.3). 

# TODO
Look at https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for ideas about hyperparameters, perplexity, convergence, etc. and https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/ for visualizations.

Also look into saving a model to disk in Gensim.

In [5]:
import numpy as np
import pandas as pd
from gensim import corpora, models, utils
import gensim
import pickle

# Read in the texts
[For test purposes one may select only the first 100 documents. Remove the hashmark (#) from the first line of the following cell if you wish to do that]

In [6]:
pickled = 'output/data_for_topic_model.p'
df = pd.read_pickle(pickled)
texts = df['lemma']

# POS-filter
The variable `posfilter` holds the last two characters of lemmatized words with allowed Part of Speech tags. If, for instance, you wish to select Verbs, Adjectives, and Nouns (in Akkadian), posfilter will be `[']n', 'aj', ']v']`. Note that one-character pos-tags need the right bracket!
The POS labels are:
* "n", #Nouns
* "v", #Verbs
* "aj", #Adjectives
* "av", #Adverbs
* "an", #Agricultural Name
* "cn", #Celestial Name
* "dn", #Divine Name
* "en", #Ethnicity Name
* "fn", #Field Name
* "gn", #Geographical Name (lands, etc.)
* "ln", #Lineage Name (ancestral clan)
* "mn", #Month Name
* "on", #Object Name
* "pn", #Personal Name
* "qn", #Quarter (of a city) Name
* "rn", #Royal Name
* "sn", #Settlement Name
* "tn", #Temple Name
* "wn", #Watercourse Name
* "yn", #Year Name
* "nu", #Numeral


In [7]:
posfilter = [']n', ']v', 'aj']
#include nouns, verbs, and adjectives, not numerals, prepositions or proper nouns
texts = [[word for word in text if word[-2:] in posfilter] for text in texts]

# Stop words

Stop words are very frequent words that are not able to distinguish between topics. This includes, for instance, prepositions - but those can also be filtered out by the POS filter. The following nouns and verbs are too frequent to contribute to the analysis. Note that this list of stop words was assembled for the SAAo corpus - another corpus may require a different list, or none at all. In a cell further below the dictionary is built - leaving out words that appear in more than 80 percent of the documents (or whatever the 'no_above' parameter is set too) making the use of a stop word list mostyly unnecessary. The only advantage of an explicit list of stop words is that it makes it possible to filter out documents or text fragments that remain with too few words to be meaningful.

The 'stoplist' cell can be omitted entirely or adapted to your purposes.

In [8]:
stoplist = [
'šarru[king]n',
'bēlu[lord]n',
'libbu[interior]n',
'muhhu[skull]n',
'ardu[slave]n',
'šulmu[completeness]n',
'šapāru[send]v',
'alāku[go]v',
'qabû[say]v',
'pānu[front]n',
'māru[son]n',
'bītu[house]n',
'epēšu[do]v',
'wabālu[bring]v',
'šakānu[put]v',
'amāru[see]v',
'bašû[exist]v',
'našû[lift]v',
'izuzzu[stand]v',
'ūmu[day]n',
'ṭābu[good]aj',
'mādu[many]aj',
'nadānu[give]v',
'tadānu[give]v',
'ṣehru[small]aj',
'mimmû[all]n',
'gimru[totality]n',
'gabbu[totality]n',
'šâlu[ask]v',
'šemû[hear]v',
'ūmu[day]n',
'awātu[word]n',
'erēbu[enter]v'
]
texts = [[word for word in text if word not in stoplist] for text in texts]


# Filter out texts that have too few words left
Identify texts that have at least 10 lemmas left and use that as a mask to filter  the list `texts` as well as the dataframe `df`. 

In [9]:
bo = [len(text)>9 for text in texts]
df = df[bo]
texts = [texts[i] for i in range(0, len(texts)) if bo[i]]

How many documents did we start with, and how many do we have left?

In [10]:
len(bo), len(df)

(4976, 3006)

# Dictionary
create the gensim Dictionary and filter for words that are too common or too rare (no_above may be set too low here).

In [11]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
## CHECK - is this done correctly?

In [12]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

# Compute the Model

Set the seed, indicate the number of topics (default set to 10) and run the model.

The visualization (section 5.3) will fail if the number of topics is higher than 25. 

In [13]:
ntopics = int(input("Number of topics: ") or 10)
if ntopics > 25:
    ntopics = 25

Number of topics:  


In [46]:
seed = 15
np.random.seed(seed)
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus, num_topics=ntopics, id2word = dictionary, passes=150)

List the top 10 words and their probabilities in all topics. Note: the topic numbers here are not the ones used in the visualizations in 5.3! (The topics are the same, but not their numbers).

In [17]:
ldamodel.show_topics(ntopics, formatted = False)

[(0,
  [('qû[unit]n', 0.039634343),
   ('ilu[god]n', 0.029594453),
   ('šikaru[beer]n', 0.024415389),
   ('karānu[vine]n', 0.022837136),
   ('šamnu[oil]n', 0.020545265),
   ('kusāpu[bread]n', 0.017754516),
   ('zamāru[sing]v', 0.016015613),
   ('dišpu[honey]n', 0.01449957),
   ('nīqu[offering]n', 0.014218154),
   ('naqû[pour-(a-libation)]v', 0.014005931)]),
 (1,
  [('nišu[people]n', 0.069575176),
   ('imēru[unit]n', 0.06602787),
   ('eqlu[field]n', 0.05771984),
   ('immeru[sheep]n', 0.04232107),
   ('ikkaru[farmer]n', 0.041318834),
   ('alpu[ox]n', 0.03420751),
   ('kirû[garden]n', 0.031347144),
   ('sinništu[woman]n', 0.027884573),
   ('tarbaṣu[courtyard]n', 0.025301298),
   ('lawû[surround]v', 0.024567386)]),
 (2,
  [('rabû[big-one]n', 0.07256891),
   ('pīhātu[responsibility]n', 0.03993749),
   ('ša-rēši[eunuch]n', 0.024973493),
   ('ša-qurbūti[close-follower]n', 0.02401254),
   ('ṣābu[people]n', 0.0203953),
   ('ēkallu[palace]n', 0.019527612),
   ('šaknu[appointee]n', 0.01871669),
 

# Document/Topic Probability
The function `get_document_topics()` will list the probability of the topics in a single document. In order to get all the topics set the argument `minimum_probability` to zero. 

In [18]:
ldamodel.get_document_topics(corpus[1], minimum_probability=0)

[(0, 0.044934496),
 (1, 0.0038660136),
 (2, 0.046452247),
 (3, 0.003865863),
 (4, 0.0038665228),
 (5, 0.0038663505),
 (6, 0.087799735),
 (7, 0.099571794),
 (8, 0.0038660106),
 (9, 0.7019109)]

# Create Document/Topic Probability Table
A Document/Topiuc probability table is a table (DataFrame), where each row represents a document and each column a topuic. Each cell has the probability of a particular topic in a particular document. The sum of each row is 1 (probability distribution).

In order to create a full Document/Topic probability table we iterate over the entire corpus with the `get_document_topics()` function. This creates a list of lists (`list_of_doctopics`) where each list represents the probability of each topic in a document. The probability is represented in a tuple (topic_number, probability). The `list_of_probabilities` preserves only the probabilities. This list of lists is transformed into a DataFrame, whith as index the index of the original DataFrame with the tokenized data.  

In [19]:
list_of_doctopics = [ldamodel.get_document_topics(corpus[i], minimum_probability=0) for i in range(len(corpus))]
list_of_probabilities = [[probability for label,probability in distribution] for distribution in list_of_doctopics]
d_t_df = pd.DataFrame(list_of_probabilities)
d_t_df = d_t_df.set_index(df.index)
d_t_df.head()

,0,1,2,3,4,5,6,7,8,9
P224378,0.009094,0.009094,0.009096,0.009096,0.009097,0.102053,0.009094,0.009095,0.009094,0.825185
P224382,0.044935,0.003866,0.046453,0.003866,0.003867,0.003866,0.087804,0.099572,0.003866,0.701906
P224383,0.003334,0.003334,0.320284,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653042
P224386,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156555,0.790102
P224388,0.006669,0.006671,0.098229,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848412


We can use the above table to find the ten highest scoring documents per topic with the pandas function 'nlargest'. First add the 'designation' as a separate column to the table.

In [24]:
d_t_df_w_desig = pd.merge(df['designation'], d_t_df, left_index=True, right_index=True)
d_t_df_w_desig

,designation,0,1,2,3,4,5,6,7,8,9
P224378,Take Over the Kingship!,0.009094,0.009094,0.009096,0.009096,0.009097,0.102053,0.009094,0.009095,0.009094,0.825185
P224382,Family Affairs,0.044935,0.003866,0.046453,0.003866,0.003867,0.003866,0.087804,0.099572,0.003866,0.701906
P224383,Šubrian King Protecting Deserters,0.003334,0.003334,0.320284,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653042
P224386,Specialists Reviving the Land,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156555,0.790102
P224388,Elamite King and the Men of Mukin-zeri,0.006669,0.006671,0.098229,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848412
P224390,Boats and Water-Skin Rafts are Well Despite an...,0.005888,0.064087,0.186909,0.005888,0.005887,0.125120,0.005888,0.005888,0.005887,0.588559
P224391,Assigning Men and Donkeys,0.005556,0.232650,0.619308,0.005557,0.005556,0.005556,0.005556,0.005556,0.005557,0.109148
P224392,Mule Express not Available,0.004001,0.004000,0.421158,0.004001,0.004000,0.004000,0.004001,0.004001,0.546836,0.004002
P224393,Fragment Referring to Boats and River Transport,0.004169,0.094350,0.004171,0.004169,0.004169,0.004169,0.004170,0.004169,0.004169,0.872293
P224395,Arabs Attack a Column of Booty,0.004002,0.004002,0.004002,0.106330,0.004001,0.004001,0.004001,0.004001,0.096847,0.768812


The following code goes through the (numbered) columns of the table which hold the probabilities of each of the topics (the columns) in each of the documents (the rows). The highest ten probabilities are selected, together with a brief descriptipon of the text (designation). 

In [25]:
doctop = []
for i in range(ntopics):
    t = d_t_df_w_desig.nlargest(10, i)[['designation', i]]
    t['topic'] = i
    t = t.rename(columns = {i :'probability'})
    doctop.append(t)
doctop_df = pd.concat(doctop, axis=0)
doctop_df

,designation,probability,topic
P336282,Unplaced Fragment of the Text of No. 69,0.976921,0
P335833,"Aššur Temple Offerings, Day 10",0.974283,0
P282529,Rituals on [Shebat 23?],0.955194,0
P425166,Fragment of a Ritual for Singer,0.952630,0
P336141,Rituals on Shebat 23-25,0.944188,0
P335668,Amounts of Grain(?) from(?) Individuals,0.943747,0
P336650,Aššur Temple Offerings,0.935713,0
P336212,Similar to No. 143,0.924999,0
P336307,Rituals on Shebat 18-22,0.921071,0
P335850,Aššur Temple Offerings,0.915630,0


# Renumber Topics
Rename the topics (columns) to start with 1, in accordance with the pyLDAvis visualization.

The LDAvis package, used in notebook 5.3, was originally written for statistical programming language R; pyLDAvis is a python wrapper. In R indexing starts with 1, rather than 0 (as is the case in Python). In order to prevent confusion, we will rename all topics according to the pyLDAvis convention.

In [26]:
topics = [i+1 for i in range(ntopics)]
d_t_df.columns = topics
d_t_df

,1,2,3,4,5,6,7,8,9,10
P224378,0.009094,0.009094,0.009096,0.009096,0.009097,0.102053,0.009094,0.009095,0.009094,0.825185
P224382,0.044935,0.003866,0.046453,0.003866,0.003867,0.003866,0.087804,0.099572,0.003866,0.701906
P224383,0.003334,0.003334,0.320284,0.003334,0.003334,0.003335,0.003334,0.003334,0.003335,0.653042
P224386,0.006667,0.006667,0.006668,0.006668,0.006669,0.006667,0.006667,0.006669,0.156555,0.790102
P224388,0.006669,0.006671,0.098229,0.006670,0.006670,0.006670,0.006670,0.006670,0.006670,0.848412
P224390,0.005888,0.064087,0.186909,0.005888,0.005887,0.125120,0.005888,0.005888,0.005887,0.588559
P224391,0.005556,0.232650,0.619308,0.005557,0.005556,0.005556,0.005556,0.005556,0.005557,0.109148
P224392,0.004001,0.004000,0.421158,0.004001,0.004000,0.004000,0.004001,0.004001,0.546836,0.004002
P224393,0.004169,0.094350,0.004171,0.004169,0.004169,0.004169,0.004170,0.004169,0.004169,0.872293
P224395,0.004002,0.004002,0.004002,0.106330,0.004001,0.004001,0.004001,0.004001,0.096847,0.768812


# Create Topic / Term table
This is a table with N rows (the number of topics) and M columns (the number of individual terms in the Dictionary). The table indicates the probability of each term in each topic.

In [27]:
topic_term = ldamodel.show_topics(ntopics, formatted=False, num_words=len(dictionary))

The object `topic_term` is a list of tuples in the format (index, \[list\]). The index is the topic number, the list is again a list of tuples in the form `(lemma, probability)` in descending order of probability. This means that the lemmas are in a different order in each of the lists.

The command `pd.DataFrame(topic_term[i][1])` creates a DataFrame with two columns for the topic `i`. The first column (column 0) has the lemmas, the second column the probabilities. By using the command `set_index(0)` the lemmas become the index (rather than a column) and this index can be used to concatenate the DataFrames (one for each topic) with the `sort = True` option. 

In [29]:
topic_term_list = [pd.DataFrame(topic_term[i][1]).set_index(0) for i in range(ntopics)]
t_t_df_ = pd.concat(topic_term_list, axis=1, ignore_index=True, sort=True)
t_t_df_.head()

,0,1,2,3,4,5,6,7,8,9
aban-bāšti[(a-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004
aban-lamassi[(a-precious-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000404,0.000014,0.000029,0.000004
aban-râmi['love'-stone]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004
abati[(meaning-unknown)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000029,0.000028
abašmû[(a-stone)]n,0.000008,0.000014,0.000014,0.000005,0.000009,0.000009,0.000008,0.000014,0.000498,0.000004


Rename the columns to start with 1, and Transpose to Topic/Term matrix.

In [ ]:
t_t_df_.columns = topics
t_t_df = t_t_df_.T
t_t_df

In [ ]:
#just checking
t_t_df['ēkallu[palace]n']

# Export Data

In [ ]:
topic_model = {'dictionary': dictionary,
                  'corpus' : corpus,
                  'ldamodel' : ldamodel,
                  't_t_df' : t_t_df,
                  'd_t_df' : d_t_df,
               'df' : df,
              'ntopics' : ntopics,
              'texts' : texts}

In [ ]:
with open('output/topic_model.p', 'wb') as w:
    pickle.dump(topic_model, w)